## Assimilation import

In [1]:
# Basic setting for Jupyter_notebook to import utils
import os
import sys

notebook_path = os.path.abspath("")
project_root = os.path.abspath(os.path.join(notebook_path, "../../"))

sys.path.append(project_root)

In [2]:
import os
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from utils import folder_utils

In [3]:
''' 

@author: Ashesh Chattopadhyay
This is a hybird SPEnKF implementation with U-STNx as the backgroud forecast model.

More details in paper: https://gmd.copernicus.org/preprints/gmd-2021-71/

The github repository contains an jupyter notebook to train the U-STNx model with different values of "x"



'''

import numpy as np
import netCDF4 as nc
import scipy.io as sio

In [4]:
### This .mat file has been generated from the ERA5 lat-lon data ####
# file=sio.loadmat('ERA_grid.mat')
# lat=file['lat']
# lon=file['lon']

lat = np.linspace(50.0, 57.75, 32)  # latitude
lon = np.linspace(-6.0, 1.875, 64)  # longitude
# 3.Define the grid
# g_lon = np.linspace(-6.0, 1.875, 64)  # longitude
# g_lat = np.linspace(50.0, 57.75, 32)  # latitude
# # gridx, gridy = np.meshgrid(gridx, gridy)

In [5]:
# Input setting
# Example usage
country = "GB"
data_folder = "data"
data_read_category = "da_test_data"
data_test_category = "test_data"
data_save_category = "assimilated_data"
output_folder = "2022_data"


In [6]:
def get_era5_list(country, data_folder, data_category, output_folder):
    input_folder_path = folder_utils.find_folder(
        country, data_folder, data_category, output_folder
    )
    nc_files = [
        f for f in os.listdir(input_folder_path) if f.endswith(".nc")
    ]
    return [
        os.path.join(input_folder_path, nc_file) for nc_file in nc_files
    ]  # list for era5 nc files path

In [7]:
########## This is the testing set #######

fileList_test=get_era5_list(country,data_folder,data_read_category,output_folder)
fileList_test

['F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_data\\GB_2022_data\\asos_kridge_2022.nc',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_data\\GB_2022_data\\era5_pressure_level_2022_regrid_filter_850.nc']

In [8]:
########### Ensure same normalization coefficient as trainig #######
M = 273.77817
sdev = 2.5819736

In [9]:
####### True data (noise free) for twin DA experiments ##########
## here can be modified 

F=nc.Dataset(fileList_test[1])
Z=np.asarray(F['t'])
TRUTH=Z

### Meshgrid for plotting ###
[qx,qy]=np.meshgrid(lon,lat)

In [10]:
##### Add noise to the truth to mimic observations####
#### Value 1 is 1*\sigma_Z. See more in paper #####
Z_rs = np.reshape(Z,[np.size(Z,0), int(np.size(Z,1)*np.size(Z,2))])
TRUTH = Z_rs
Z_rs = (Z_rs-M)/sdev
TRUTH = (TRUTH-M)/sdev
noise=1 # modify here

In [11]:
for k in range(1,np.size(Z_rs,0)):
    Z_rs[k-1,:]=Z_rs[k-1,:]+np.random.normal(0, noise, 2048)
print('length of initial condition',len(Z_rs[0,:]))

length of initial condition 2048


In [11]:
#### SPNEKF implementation following Tyrus Berry's implementation ######

def ENKF(x, n, P ,Q, R, obs, model, u_ensemble):
    print("obs\n")
    obs=np.reshape(obs,[n,1]) 
    print("x\n")
    x=np.reshape(x,[n,1])
    print("USV\n")
    [U,S,V]=np.linalg.svd(P)
    print("D\n")
    D=np.zeros([n,n])
    print("diagonal\n")
    np.fill_diagonal(D,S)
    print("sqrtP\n")
    sqrtP=np.dot(np.dot(U,np.sqrt(D)),U)
    print("ens1\n")
    ens=np.zeros([n,2*n])
    print("ens2\n")
    ens[:,0:n]=np.tile(x,(1,n)) + sqrtP
    print("ens3\n")
    ens[:,n:]=np.tile(x,(1,n)) - sqrtP
    ## forecasting step,dummy model

    for k in tqdm(range(0, np.size(ens,1))):

       u =  model.predict(np.reshape(ens[:,k],[1, 32, 64, 1]))

       u_ensemble[:,k]=np.reshape(u,(32*64,))



    ############################
    print("x_prior\n")
    x_prior = np.reshape(np.mean(u_ensemble,1),[n,1])
    print('shape pf x_prior',np.shape(x_prior))
    print('shape pf obs',np.shape(obs))
    print("cf_ens\n")
    cf_ens = ens - np.tile(x_prior,(1,2*n))
    print("P_prior\n")
    P_prior = np.dot(cf_ens,np.transpose(cf_ens))/(2*n - 1)+Q
    print("h_ens\n")
    h_ens = ens
    print("y_prior\n")
    y_prior=np.reshape(np.mean(h_ens,1),[n,1])
    ch_ens = h_ens - np.tile(y_prior,(1,2*n))
    print('shape pf y_prior',np.shape(y_prior))
    print("P_y\n")
    P_y = np.dot(ch_ens, np.transpose(ch_ens))/(2*n-1) + R
    print("P_xy\n")
    P_xy = np.dot(cf_ens, np.transpose(ch_ens)) /(2*n-1)
    print("K\n")
    K = np.dot(P_xy,np.linalg.inv(P_y))
    print("P\n")
    P = P_prior - np.dot(np.dot(K,P_y),np.transpose(K))
    print("x\n")
    x = x_prior + np.dot(K,(obs-y_prior))

    return x, P

In [12]:
def get_initial_weights(output_size):
    b = np.zeros((2, 3), dtype='float32')
    b[0, 0] = 1
    b[1, 1] = 1
    W = np.zeros((output_size, 6), dtype='float32')
    weights = [W, b.flatten()]
    return weights

In [13]:
from keras import backend as K
from keras.engine.topology import Layer

if K.backend() == 'tensorflow':
    import tensorflow as tf

    def K_meshgrid(x, y):
        return tf.meshgrid(x, y)

    def K_linspace(start, stop, num):
        return tf.linspace(start, stop, num)

else:
    raise Exception("Only 'tensorflow' is supported as backend")


class BilinearInterpolation(Layer):
    """Performs bilinear interpolation as a keras layer
    References
    ----------
    [1]  Spatial Transformer Networks, Max Jaderberg, et al.
    [2]  https://github.com/skaae/transformer_network
    [3]  https://github.com/EderSantana/seya
    """

    def __init__(self, output_size, **kwargs):
        self.output_size = output_size
        super(BilinearInterpolation, self).__init__(**kwargs)

    def get_config(self):
        return {
            'output_size': self.output_size,
        }

    def compute_output_shape(self, input_shapes):
        height, width = self.output_size
        num_channels = input_shapes[0][-1]
        return (None, height, width, num_channels)

    def call(self, tensors, mask=None):
        X, transformation = tensors
        output = self._transform(X, transformation, self.output_size)
        return output

    def _interpolate(self, image, sampled_grids, output_size):

        batch_size = K.shape(image)[0]
        height = K.shape(image)[1]
        width = K.shape(image)[2]
        num_channels = K.shape(image)[3]

        x = K.cast(K.flatten(sampled_grids[:, 0:1, :]), dtype='float32')
        y = K.cast(K.flatten(sampled_grids[:, 1:2, :]), dtype='float32')

        x = .5 * (x + 1.0) * K.cast(width, dtype='float32')
        y = .5 * (y + 1.0) * K.cast(height, dtype='float32')

        x0 = K.cast(x, 'int32')
        x1 = x0 + 1
        y0 = K.cast(y, 'int32')
        y1 = y0 + 1

        max_x = int(K.int_shape(image)[2] - 1)
        max_y = int(K.int_shape(image)[1] - 1)

        x0 = K.clip(x0, 0, max_x)
        x1 = K.clip(x1, 0, max_x)
        y0 = K.clip(y0, 0, max_y)
        y1 = K.clip(y1, 0, max_y)

        pixels_batch = K.arange(0, batch_size) * (height * width)
        pixels_batch = K.expand_dims(pixels_batch, axis=-1)
        flat_output_size = output_size[0] * output_size[1]
        base = K.repeat_elements(pixels_batch, flat_output_size, axis=1)
        base = K.flatten(base)

        # base_y0 = base + (y0 * width)
        base_y0 = y0 * width
        base_y0 = base + base_y0
        # base_y1 = base + (y1 * width)
        base_y1 = y1 * width
        base_y1 = base_y1 + base

        indices_a = base_y0 + x0
        indices_b = base_y1 + x0
        indices_c = base_y0 + x1
        indices_d = base_y1 + x1

        flat_image = K.reshape(image, shape=(-1, num_channels))
        flat_image = K.cast(flat_image, dtype='float32')
        pixel_values_a = K.gather(flat_image, indices_a)
        pixel_values_b = K.gather(flat_image, indices_b)
        pixel_values_c = K.gather(flat_image, indices_c)
        pixel_values_d = K.gather(flat_image, indices_d)

        x0 = K.cast(x0, 'float32')
        x1 = K.cast(x1, 'float32')
        y0 = K.cast(y0, 'float32')
        y1 = K.cast(y1, 'float32')

        area_a = K.expand_dims(((x1 - x) * (y1 - y)), 1)
        area_b = K.expand_dims(((x1 - x) * (y - y0)), 1)
        area_c = K.expand_dims(((x - x0) * (y1 - y)), 1)
        area_d = K.expand_dims(((x - x0) * (y - y0)), 1)

        values_a = area_a * pixel_values_a
        values_b = area_b * pixel_values_b
        values_c = area_c * pixel_values_c
        values_d = area_d * pixel_values_d
        return values_a + values_b + values_c + values_d

    def _make_regular_grids(self, batch_size, height, width):
        # making a single regular grid
        x_linspace = K_linspace(-1., 1., width)
        y_linspace = K_linspace(-1., 1., height)
        x_coordinates, y_coordinates = K_meshgrid(x_linspace, y_linspace)
        x_coordinates = K.flatten(x_coordinates)
        y_coordinates = K.flatten(y_coordinates)
        ones = K.ones_like(x_coordinates)
        grid = K.concatenate([x_coordinates, y_coordinates, ones], 0)

        # repeating grids for each batch
        grid = K.flatten(grid)
        grids = K.tile(grid, K.stack([batch_size]))
        return K.reshape(grids, (batch_size, 3, height * width))

    def _transform(self, X, affine_transformation, output_size):
        batch_size, num_channels = K.shape(X)[0], K.shape(X)[3]
        transformations = K.reshape(affine_transformation,
                                    shape=(batch_size, 2, 3))
        # transformations = K.cast(affine_transformation[:, 0:2, :], 'float32')
        regular_grids = self._make_regular_grids(batch_size, *output_size)
        sampled_grids = K.batch_dot(transformations, regular_grids)
        interpolated_image = self._interpolate(X, sampled_grids, output_size)
        new_shape = (batch_size, output_size[0], output_size[1], num_channels)
        interpolated_image = K.reshape(interpolated_image, new_shape)
        return interpolated_image

Using TensorFlow backend.
D:\anaconda3\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\py37\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Fu

In [14]:
import tensorflow
import keras.backend as K
#from data_manager import ClutteredMNIST
#from visualizer import plot_mnist_sample
#from visualizer import print_evaluation
#from visualizer import plot_mnist_grid
import netCDF4
import numpy as np
from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
import keras
from keras.callbacks import History
history = History()

import keras
from keras.layers import Conv2D, Conv2DTranspose, Cropping2D, Concatenate, ZeroPadding2D
from keras.models import load_model

__version__ = 0.1


#### This is the circular convolution function. With/Without doesn't make much difference. If training is done with CConv2D then replace Convolution2D with CCvonv2D else leave it like this  #####
def CConv2D(filters, kernel_size, strides=(1, 1), activation='linear', padding='valid', kernel_initializer='glorot_uniform', kernel_regularizer=None):
    def CConv2D_inner(x):
        # padding (see https://www.tensorflow.org/api_guides/python/nn#Convolution)
        in_height = int(x.get_shape()[1])
        in_width = int(x.get_shape()[2])

        if (in_height % strides[0] == 0):
            pad_along_height = max(kernel_size[0] - strides[0], 0)
        else:
            pad_along_height = max(
                kernel_size[0] - (in_height % strides[0]), 0)
        if (in_width % strides[1] == 0):
            pad_along_width = max(kernel_size[1] - strides[1], 0)
        else:
            pad_along_width = max(kernel_size[1] - (in_width % strides[1]), 0)

        pad_top = pad_along_height // 2
        pad_bottom = pad_along_height - pad_top
        pad_left = pad_along_width // 2
        pad_right = pad_along_width - pad_left

        # left and right side for padding
        pad_left = Cropping2D(cropping=((0, 0), (in_width-pad_left, 0)))(x)
        pad_right = Cropping2D(cropping=((0, 0), (0, in_width-pad_right)))(x)

        # add padding to incoming image
        conc = Concatenate(axis=2)([pad_left, x, pad_right])

        # top/bottom padding options
        if padding == 'same':
            conc = ZeroPadding2D(padding={'top_pad': pad_top,
                                          'bottom_pad': pad_bottom})(conc)
        elif padding == 'valid':
            pass
        else:
            raise Exception('Padding "{}" does not exist!'.format(padding))

        # perform the circular convolution
        cconv2d = Conv2D(filters=filters, kernel_size=kernel_size,
                         strides=strides, activation=activation,
                         padding='valid',
                         kernel_initializer=kernel_initializer,
                         kernel_regularizer=kernel_regularizer)(conc)

        # return circular convolution layer
        return cconv2d
    return CConv2D_inner

from keras.layers import Input
from keras.models import Model
from keras.layers import Activation
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Dense

# from utils import get_initial_weights
# from bilinear_interpolation_1x import BilinearInterpolation

In [15]:
##### Load model. DO not train. #####
def stn(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
    inputs = Input(shape=input_shape)
    conv1 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(pool2)
#     conv3 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv3)


    conv5 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv3)
#     conv5 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv5)
    
    locnet = Flatten()(conv5)
    locnet = Dense(500)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(200)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(100)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(50)(locnet)
    locnet = Activation('relu')(locnet)
    weights = get_initial_weights(50)
    locnet = Dense(6, weights=weights)(locnet)
    x = BilinearInterpolation(sampling_size)([inputs, locnet])


    up6 = keras.layers.Concatenate(axis=-1)([Convolution2D(32, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(x)), conv2])
    conv6 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv6)

    up7 = keras.layers.Concatenate(axis=-1)([Convolution2D(32, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(conv6)), conv1])
    conv7 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv7)



    conv10 = Convolution2D(1, 5, 5, activation='linear',border_mode='same')(conv7)

    model = Model(input=inputs, output=conv10)



    return model

In [16]:
# Input setting
# Example usage
country = "GB"
data_folder = "data"
data_read_category = "da_test_data"
data_test_category = "test_data"
data_save_category = "assimilated_data"
output_folder = "2022_weight_data"

In [17]:
def get_weight_list(country, data_folder, data_category, output_folder):
    input_folder_path = folder_utils.create_folder(
        country, data_folder, data_category, output_folder
    )
    nc_files = [
        f for f in os.listdir(input_folder_path) if f.endswith(".h5")
    ]
    return [
        os.path.join(input_folder_path, nc_file) for nc_file in nc_files
    ]  # list for era5 nc files path

In [18]:
weight_list = get_weight_list(country,data_folder,data_read_category,output_folder)

Folder 'F:\JuPyterNotebook\irp_ww721_bakcup\data\da_test_data\2022_weight_data\GB_2022_weight_data' created successfully.


In [19]:
weight_list

['F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_weight_data\\GB_2022_weight_data\\best_weights_lead12.h5',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_weight_data\\GB_2022_weight_data\\best_weights_lead12_2.h5',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_weight_data\\GB_2022_weight_data\\best_weights_lead12_3.h5',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_weight_data\\GB_2022_weight_data\\best_weights_lead12_4.h5',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_weight_data\\GB_2022_weight_data\\best_weights_lead12_5.h5',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_weight_data\\GB_2022_weight_data\\best_weights_lead12_6.h5',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_weight_data\\GB_2022_weight_data\\best_weights_lead12_7.h5',
 'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_weight_data\\GB_2022_weight_data\\best_weights_

In [20]:
best_weight = weight_list[5]

In [21]:
model = stn()
model.load_weights(best_weight) 
### This code performs DA at every 24 hrs with a model that is forecasting every hour. So lead will always be 1 ######

Instructions for updating:
Colocations handled automatically by placer.


D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  after removing the cwd from sys.path.
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  """
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  if __name__ == "__main__":
D:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  if sys.path[0]

## Calculation

In [22]:
import scipy.io as sio
import numpy as np
import xarray as xr
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from tqdm import tqdm

In [23]:
era5_data = fileList_test[1]
era5_data 

'F:\\JuPyterNotebook\\irp_ww721_bakcup\\data\\da_test_data\\2022_data\\GB_2022_data\\era5_pressure_level_2022_regrid_filter_850.nc'

In [38]:
random_seed = 42
np.random.seed(random_seed)

# Set time step
time_step = 120
dt = 6
steps = time_step // dt
count = 0

In [39]:
import netCDF4 as nc  # py37

In [40]:
# Load data
era5_value =netCDF4.Dataset(era5_data)

In [41]:
era5_t = np.asarray(era5_value["t"])
era5_t.shape

(8760, 32, 64)

In [42]:
# Select 50 initial conditions randomly
initial_conditions = 50
random_indices = np.random.choice(
    era5_t.shape[0] - 240,
    size=initial_conditions,
    replace=False,  # -240 to ensure enough range for prediction
)

pred_data = np.zeros([initial_conditions, steps, 32, 64])


In [43]:
random_indices

array([7499, 7310, 2571, 1084,  856, 4081, 2131, 1385, 4959, 3375, 4507,
       1670, 3617, 1623, 6847, 7028,  315, 3519, 6383, 8046, 4461, 4409,
       2111, 1057,  868,  700, 7690, 6716, 4560, 4015, 7070, 7327, 3668,
       3949, 8496, 1183,  794, 5794, 2487, 6686, 2802, 5962, 2483,  789,
        233, 4695,  429, 2614, 5686, 1281])

In [44]:
pred_data.shape

(50, 20, 32, 64)

In [45]:
for idx, start_t in tqdm(enumerate(random_indices)):
    current_data = era5_t[start_t].reshape(1, 32, 64, 1)
    for s in range(steps):
        # Using the current data to predict the data for 24 hours later
        pred_data[idx, s] = model.predict(current_data).squeeze()
        # The predicted data is the current data for the next iteration
        current_data = pred_data[idx, s].reshape(1, 32, 64, 1)

50it [00:23,  2.12it/s]


In [46]:
pred_data.shape

(50, 20, 32, 64)

In [47]:
# Calculate accuracy
acc_values = []
for i in tqdm(range(initial_conditions)):
    for s in range(steps):
        actual_data = era5_t[random_indices[i] + s * dt]
        predicted_data = pred_data[i, s]

        correlation, _ = pearsonr(actual_data.flatten(), predicted_data.flatten())
        acc_values.append(correlation)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 27.82it/s]


In [48]:
tt = np.mean(acc_values)

In [49]:
tt

-0.04746939211496702

In [50]:
acc_values

[0.43382065439902645,
 0.5382613782793364,
 -0.1477817211645552,
 -0.09047264164290915,
 -0.3084115067874335,
 -0.46245111272081607,
 -0.7886719094854028,
 -0.600235716871462,
 -0.13299312567537813,
 -0.13846444393783414,
 -0.6417436289700011,
 -0.29983715333358213,
 0.4699766788210823,
 0.2302727987064994,
 0.3703579288782894,
 0.5752876848597315,
 0.8111433990563455,
 0.6773947195372136,
 0.4294019267505129,
 0.15319854392624888,
 0.49254047291791425,
 0.19761896579895966,
 0.36217261133806067,
 0.17033620967512972,
 0.24436458445392467,
 -0.07381896397809626,
 -0.5147746156869716,
 -0.11673728565665045,
 -0.5364915486006894,
 0.07252376343270964,
 -0.051623999221900196,
 -0.041338541898971466,
 -0.0821622787826248,
 -0.4152917417363947,
 -0.2621119705132494,
 -0.3367506601210079,
 -0.0382556163763909,
 0.033120378335521436,
 -0.532732932277114,
 -0.6274216879835409,
 0.4459726355909158,
 0.24255996184773262,
 0.3901193386569039,
 0.504687192876065,
 -0.4149074975197896,
 0.552326918

In [56]:
len(acc_values)

500

In [60]:
def calculate_acc(era5_data, stn_model_data, model_weight):
    # acc_value = 1 - np.mean(np.abs(input_value))

    # Load model
    model = stn()
    model.load_weights(model_weight)

    random_seed = 42
    np.random.seed(random_seed)

    # Set time step
    time_step = 240
    dt = 24
    steps = time_step // dt
    count = 0

    # Load data

    era5_value = xr.open_dataset(era5_data)
    era5_t = np.asarray(era5_value["t"])

    # Select 50 initial conditions randomly
    initial_conditions = 50
    random_indices = np.random.choice(
        era5_t.shape[0] - 240,
        size=initial_conditions,
        replace=False,  # -240 to ensure enough range for prediction
    )

    pred_data = np.zeros([initial_conditions, steps, 32, 64])

    for idx, start_t in tqdm(enumerate(random_indices)):
        current_data = era5_t[start_t].reshape(1, 32, 64, 1)
        for s in range(steps):
            # Using the current data to predict the data for 24 hours later
            pred_data[idx, s] = model.predict(current_data).squeeze()
            # The predicted data is the current data for the next iteration
            current_data = pred_data[idx, s].reshape(1, 32, 64, 1)

    # Calculate accuracy
    acc_values = []
    for i in tqdm(range(initial_conditions)):
        for s in range(steps):
            actual_data = era5_t[random_indices[i] + s * dt]
            predicted_data = pred_data[i, s]

            correlation, _ = pearsonr(actual_data.flatten(), predicted_data.flatten())
            acc_values.append(correlation)

    return acc_values

In [37]:
np.mean(acc_values)

-0.069554518191299